In [1]:
import pandas as pd
import numpy as np
import time
from datetime import timedelta,datetime

In [3]:
def data_split(df,well_id_int=6110299100,param_id_list=[188,200],package_size=50000, leftcut=5000):
    
    df=df.query('well_id == @well_id_int and param_id in @param_id_list')
    del df['well_id']
    # Устанавливаем tm_time как индекс
    df.set_index('tm_time', inplace=True)

    # Отсортируем по индексу
    df.sort_index(inplace=True)

    # Преобразование с помощью pivot
    df = df.pivot(columns='param_id', values='tm_value')

    # убираем верхний индекс param_id
    #df.columns.rename(None, inplace=True)

    
    # Интерполирование к частоте в 1 секунду
    df = df.resample('1s').mean().interpolate(method='linear', limit_direction='both')
    
    packages = []
    current_packet = pd.DataFrame()  # Изначально пустой пакет
    num_points = len(df)

    # Идем по ряду с шагом package_size
    for i in range(0, num_points, package_size):
        # Берем кусок данных длиной package_size
        package = df[i:i + package_size]

         # Добавляем новые точки к текущему пакету
        current_packet = pd.concat([current_packet[leftcut:], package])

        # Добавляем пакет в список
        packages.append(current_packet)

    return packages    

In [17]:
def calculate_deviations(list_of_df, dict_values = {188: [50,150], 200: [20, 55]}):
    
    # Создаем пустой список 
    result_list=[]
    
    for df in list_of_df:
        
        # Создаем пустой датафрейм для результатов
        result_df = pd.DataFrame(index=df.index)
        
        # Создаем пустой датафрейм статистик
        stats_dict = {}
        
        for col in df.columns:
            
            # Выбираем конкретный столбец
            current_column = df[col]

            # Выбираем соответствующий список значений для текущего столбца
            set_min_value, set_max_value = dict_values.get(col)
            
            # Вычисление среднего, медианы, квартилей и дисперсии для данного столбца
            mean_value = current_column.mean()
            median_value = current_column.median()
            quartiles = current_column.quantile([0.25, 0.75])
            std_dev = current_column.std()
            
            # Добавляем статистики в словарь
            stats_dict[col] = {
            'mean': mean_value,
            'Q1': quartiles.loc[0.25],
            'median': median_value,
            'Q3': quartiles.loc[0.75],
            'std': std_dev,
            'set_min': set_min_value,
            'set_max': set_max_value
                }
            # Столбец данных
            result_df[f'{col}_Value'] = current_column
            
            # Отклонения от среднего
            result_df[f'{col}_Deviation from Mean'] = current_column - mean_value
        
            # Отклонения от первого квартиля
            result_df[f'{col}_Deviation from Q1'] = current_column - quartiles.loc[0.25]
        
            # Отклонения от медианы
            result_df[f'{col}_Deviation from Median'] = current_column - median_value
        
            # Отклонения от третьего квартиля
            result_df[f'{col}_Deviation from Q3'] = current_column - quartiles.loc[0.75]
        
            # Отклонения от предыдущего значения
            result_df[f'{col}_Deviation from Previous Value'] = current_column.diff()

            # Отклонения от min значения
            result_df[f'{col}_Deviation from Set Min Value'] = current_column - set_min_value

            # Отклонения от max значения
            result_df[f'{col}_Deviation from Set Max Value'] = current_column - set_max_value
    
        result_list.append([result_df,stats_dict])
        
    return result_list
    



In [19]:
df1=pd.read_pickle('ncrptd.pkl')
df1.reset_index(inplace=True)

df=pd.DataFrame(columns=['well_id', 'param_id', 'tm_time','tm_value'])
df['well_id']=df1['wd']+2024
df['param_id']=df1['pd']+1703
df['tm_time']=df1['tt']+timedelta(days=10000)
df['tm_value']=df1['tv']/1.1

df

,well_id,param_id,tm_time,tm_value
0,6110299100,220,2024-04-01 13:14:08,0.0
1,6110299100,220,2024-04-01 13:14:38,0.0
2,6110299100,220,2024-04-01 13:15:08,0.0
3,6110299100,220,2024-04-01 13:15:38,0.0
4,6110299100,220,2024-04-01 13:16:08,0.0
...,...,...,...,...
15497320,2860194100,401,2024-04-01 23:58:03,35.7
15497321,2860194100,401,2024-04-01 23:58:33,35.1
15497322,2860194100,401,2024-04-01 23:59:03,0.0
15497323,2860194100,401,2024-04-01 23:59:04,0.0


In [21]:
list_of_dfs = data_split(df,well_id_int=6110299100,param_id_list=[188,200],package_size=5000, leftcut=0)

C:\Users\Pro10\AppData\Local\Temp\ipykernel_7736\1239302328.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_index(inplace=True)


In [23]:
result_df = calculate_deviations(list_of_dfs, dict_values = {188: [50,200], 200: [20, 55]})


In [25]:
result_df[-1][0]

,188_Value,188_Deviation from Mean,188_Deviation from Q1,188_Deviation from Median,188_Deviation from Q3,188_Deviation from Previous Value,188_Deviation from Set Min Value,188_Deviation from Set Max Value,200_Value,200_Deviation from Mean,200_Deviation from Q1,200_Deviation from Median,200_Deviation from Q3,200_Deviation from Previous Value,200_Deviation from Set Min Value,200_Deviation from Set Max Value
tm_time,,,,,,,,,,,,,,,,
2024-04-01 00:00:03,30.0,-0.338819,0.0,0.0,-1.0,NaN,-20.0,-170.0,74.000000,49.312429,74.000000,74.000000,-0.166667,NaN,54.000000,19.000000
2024-04-01 00:00:04,30.0,-0.338819,0.0,0.0,-1.0,0.0,-20.0,-170.0,74.033333,49.345762,74.033333,74.033333,-0.133333,0.033333,54.033333,19.033333
2024-04-01 00:00:05,30.0,-0.338819,0.0,0.0,-1.0,0.0,-20.0,-170.0,74.066667,49.379096,74.066667,74.066667,-0.100000,0.033333,54.066667,19.066667
2024-04-01 00:00:06,30.0,-0.338819,0.0,0.0,-1.0,0.0,-20.0,-170.0,74.100000,49.412429,74.100000,74.100000,-0.066667,0.033333,54.100000,19.100000
2024-04-01 00:00:07,30.0,-0.338819,0.0,0.0,-1.0,0.0,-20.0,-170.0,74.133333,49.445762,74.133333,74.133333,-0.033333,0.033333,54.133333,19.133333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01 23:59:36,30.0,-0.338819,0.0,0.0,-1.0,0.0,-20.0,-170.0,74.000000,49.312429,74.000000,74.000000,-0.166667,0.000000,54.000000,19.000000
2024-04-01 23:59:37,30.0,-0.338819,0.0,0.0,-1.0,0.0,-20.0,-170.0,74.000000,49.312429,74.000000,74.000000,-0.166667,0.000000,54.000000,19.000000
2024-04-01 23:59:38,30.0,-0.338819,0.0,0.0,-1.0,0.0,-20.0,-170.0,74.000000,49.312429,74.000000,74.000000,-0.166667,0.000000,54.000000,19.000000


In [27]:
result_df[-1][1]

{188: {'mean': 30.33881891222302,
  'Q1': 29.999999999999996,
  'median': 29.999999999999996,
  'Q3': 31.0,
  'std': 0.8884281126652277,
  'set_min': 50,
  'set_max': 200},
 200: {'mean': 24.687570909259303,
  'Q1': 0.0,
  'median': 0.0,
  'Q3': 74.16666666666667,
  'std': 34.95802376631494,
  'set_min': 20,
  'set_max': 55}}